Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'

REPO_ROOT = "/home/leon/models/NeuroFlame"
pal = sns.color_palette("tab10")
```

Imports
=======

``` ipython
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from scipy.stats import binned_statistic
```

``` ipython
import sys
sys.path.insert(0, '../../../')

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump, circcvl, decode_bump_torch
from src.lr_utils import masked_normalize, clamp_tensor, normalize_tensor
from src.utils import clear_cache
```

``` ipython
import pickle as pkl
import os

def pkl_save(obj, name, path="."):
    os.makedirs(path, exist_ok=True)
    destination = path + "/" + name + ".pkl"
    print("saving to", destination)
    pkl.dump(obj, open(destination, "wb"))


def pkl_load(name, path="."):
    source = path + "/" + name + '.pkl'
    print('loading from', source)
    return pkl.load(open( source, "rb"))

```

Helpers
=======

``` ipython
def map2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return np.where(angles > np.pi, angles - 2 * np.pi, angles)

def map2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return np.where(angles < 0, angles + 2 * np.pi, angles)
```

``` ipython
def maptens2center(angles):
    """Map angles from [0, 2π] to [-π, π] using PyTorch tensors."""
    return torch.where(angles > torch.pi, angles - 2 * torch.pi, angles)

def maptens2pos(angles):
    """Map angles from [-π, π] to [0, 2π] using PyTorch tensors."""
    return torch.where(angles < 0, angles + 2 * torch.pi, angles)
```

``` ipython
def add_vlines(model, ax=None):

    if ax is None:
        for i in range(len(model.T_STIM_ON)):
            plt.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)
    else:
        for i in range(len(model.T_STIM_ON)):
            ax.axvspan(model.T_STIM_ON[i], model.T_STIM_OFF[i], alpha=0.25)

```

``` ipython
import torch
import numpy as np

def generate_weighted_phase_samples(N_BATCH, angles, preferred_angle, sigma):
    # Convert angles list to a tensor
    angles_tensor = torch.tensor(angles)

    # Calculate Gaussian probability distribution centered at preferred_angle
    probs = np.exp(-0.5 * ((angles - preferred_angle) / sigma) ** 2)
    probs /= probs.sum()  # Normalize to get probabilities

    # Create a categorical distribution from the computed probabilities
    distribution = torch.distributions.Categorical(torch.tensor(probs))

    # Sample from the distribution
    indices = distribution.sample((N_BATCH,))

    # Map indices to angles and reshape to (N_BATCH, 1)
    phase_samples = angles_tensor[indices].reshape(N_BATCH, 1)

    return phase_samples
```

``` ipython
import torch
import numpy as np
import matplotlib.pyplot as plt

def continuous_biased_phases(N_BATCH, preferred_angle, sigma):
    # Generate samples from a normal distribution using PyTorch
    phase_samples = torch.normal(mean=preferred_angle, std=sigma, size=(N_BATCH, 1))

    # Normalize angles to the range [0, 360)
    phase_samples = phase_samples % 360

    return phase_samples
```

``` ipython
import torch
import numpy as np
import matplotlib.pyplot as plt

def continuous_bimodal_phases(N_BATCH, preferred_angle, sigma):
    # Sample half from preferred_angle and half from preferred_angle + 180
    half_batch = N_BATCH // 2

    # Sample from preferred_angle
    samples_1 = torch.normal(mean=preferred_angle, std=sigma, size=(half_batch, 1))

    # Sample from preferred_angle + 180
    samples_2 = torch.normal(mean=(preferred_angle + 180) % 360, std=sigma, size=(N_BATCH - half_batch, 1))

    # Combine samples and wrap around 360
    phase_samples = torch.cat((samples_1, samples_2), dim=0) % 360

    return phase_samples
```

``` ipython
def continuous_bimodal_phases(N_BATCH, reference, sigma):

    ref_copy = reference.clone()
    idx = torch.rand(N_BATCH, 1) > 0.5
    ref_copy[idx] = (reference[idx] + 180.0) % 360.0
    samples = ref_copy + sigma * torch.randn(N_BATCH, 1)

    return samples % 360.0
```

Model
=====

``` ipython
kwargs = {
    'GAIN': 1.0,
    'DURATION': 12.0,
    'T_STEADY': 2,
    'T_STIM_ON': [1.0, 5.0, 7.0, 11.0],
    'T_STIM_OFF': [2.0, 6.0, 8.0, 12.0],
    'I0': [1.0, -2.0, 1.0, -2.0],
    'PHI0': [180.0, 180, 180, 180],
    'SIGMA0': [1.0, 0.0, 1.0, 0.0],
    'RANDOM_DELAY': 0,
    'MIN_DELAY': 2,
    'MAX_DELAY': 5,
    'TAU_FAC': [2],
    'IF_ADAPT': 1,
    'A_ADAPT': 0.5,
    'TAU_ADAPT': 100.0,
    'IF_FF_ADAPT': 0,
    'A_FF_ADAPT': 0.3,
    'TAU_FF_ADAPT': 100.0,
}
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"
conf_name = "train_odr_EI.yml"
DEVICE = 'cuda:1'
# seed = np.random.randint(0, 1e6)

seed = 3
print('seed', seed)
```

``` ipython
N_BATCH = 128 * 6
print(N_BATCH)
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=N_BATCH, **kwargs)
```

``` ipython
model_state_dict = torch.load('../models/odr/odr_%d.pth' % seed)
model.load_state_dict(model_state_dict);
model.eval();
```

Simulating Consecutive Trials
=============================

``` ipython
reference = 360.0 * torch.rand(N_BATCH, 1)
sigma = 45
```

``` ipython
model.N_BATCH = N_BATCH

model.PHI0 = torch.zeros(size=(N_BATCH, len(model.I0), 1), device=DEVICE, dtype=torch.float)
model.PHI0[:, 0] = continuous_bimodal_phases(N_BATCH, reference, sigma)
model.PHI0[:, 2] = continuous_bimodal_phases(N_BATCH, reference, sigma)

print(reference.shape, model.PHI0.shape)

with torch.no_grad():
    ff_input = model.init_ff_input()
    rates_tensor = model.forward(ff_input=ff_input)
    del ff_input
    clear_cache()

print(rates_tensor.shape)
```

``` ipython
num_epochs = 49

rates_list = [rates_tensor.cpu().detach()]
phi0_list = [model.PHI0.cpu().detach()]
thresh_list = [model.thresh_last.cpu().detach()]

for epoch in tqdm(range(num_epochs)):
    # here we run two consecutive trial at a time (epoch)
    with torch.no_grad():
        model.PHI0 = torch.zeros(size=(N_BATCH, len(model.I0), 1), device=DEVICE, dtype=torch.float)
        model.PHI0[:, 0] = continuous_bimodal_phases(N_BATCH, reference, sigma)
        model.PHI0[:, 2] = continuous_bimodal_phases(N_BATCH, reference, sigma)

        ff_input = model.init_ff_input()
        rates = model.forward(ff_input=ff_input, IF_INIT=0)

        phi0_list.append(model.PHI0.cpu().detach())
        rates_list.append(rates.cpu().detach())
        thresh_list.append(model.thresh_last.cpu().detach())

        del ff_input, model.PHI0, rates
        clear_cache()

rates_list = np.stack(rates_list)
thresh_list = np.stack(thresh_list)
phi0_list = np.stack(phi0_list)

print('rates', rates_list.shape, 'thresh', thresh_list.shape, 'phi0', phi0_list.shape, reference.shape)
```

``` ipython
plt.hist(phi0_list[:, 0, 0, 0] * 180 / np.pi, bins=20)
plt.show()
```

``` ipython
plt.imshow(thresh_list[:,0, :750].T, aspect='auto', cmap='jet', vmin=0)
plt.xlabel('Trial Pair')
plt.ylabel('Neuron #')
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
ax[0].scatter(rates_list[0, 0, -1], rates_list[-1, 0, -1])
ax[1].scatter(thresh_list[0, 0], thresh_list[-1, 0])

plt.show()
```

``` ipython
plt.plot(thresh_list[:, 0, :10])
plt.xlabel('Trial Pair')
plt.ylabel('Threshold')
plt.show()
```

Errors
======

Errors
------

``` ipython
n_half = num_epochs // 4
# n_half = 1
phi0_ini =  phi0_list[:n_half]
phi0_last = phi0_list[-n_half:]

print(phi0_ini.shape, phi0_last.shape, reference.shape, rates_list.shape)
```

``` ipython
_, _, phi_ini = decode_bump_torch(rates_list[:n_half, ...], axis=-1)
print(phi_ini.shape)
```

``` ipython
_, _, phi_last = decode_bump_torch(rates_list[-n_half:, ...], axis=-1)
print(phi_last.shape)
```

``` ipython
def get_rel_tar_error(phi, phi0, reference):
    target_loc = phi0[:, :, 2]  * 180.0 / np.pi

    rel_loc = phi0[:, :, 0] - phi0[:, :, 2]
    rel_loc = (rel_loc + np.pi) % (2 * np.pi) - np.pi
    rel_loc *= 180 / np.pi

    ref_loc = reference[np.newaxis] - phi0[:, :, 2]
    ref_loc = (ref_loc + np.pi) % (2 * np.pi) - np.pi
    ref_loc *= 180 / np.pi

    error_curr = phi - phi0[:, :, 2]
    error_curr = (error_curr + np.pi) % (2 * np.pi) - np.pi
    error_curr *= 180 / np.pi

    error_prev = phi - phi0[:, :,0]
    error_prev = (error_prev + np.pi) % (2 * np.pi) - np.pi
    error_prev *= 180 / np.pi

    errors = np.stack((error_prev, error_curr), 1)

    return np.vstack(target_loc), np.vstack(rel_loc), np.vstack(ref_loc), errors
```

``` ipython
def get_end_point(model, errors):

    stim_start_idx = ((model.start_indices - model.N_STEADY) / model.N_WINDOW).to(int).cpu().numpy()

    end_point = []
    for i, j in enumerate([1, 3]):
        end_ = []
        for k in range(errors.shape[2]):
            idx = stim_start_idx[j][k]
            end_.append(errors[:, i, k, idx])

        end_point.append(end_)

    return np.vstack(np.array(end_point).T).T
```

``` ipython
targ_ini, rel_ini, ref_ini, errors_ini = get_rel_tar_error(phi_ini, phi0_ini, reference * np.pi / 180.0)
targ_last, rel_last, ref_last, errors_last = get_rel_tar_error(phi_last, phi0_last, reference * np.pi / 180.0)
```

``` ipython
print(ref_ini.shape, rel_ini.shape, errors_ini.shape)
```

``` ipython
end_point_ini = get_end_point(model, errors_ini)
end_point_last = get_end_point(model, errors_last)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

ax[0].hist(reference[:, 0])
ax[1].hist(end_point_ini[1], bins='auto')
ax[2].hist(end_point_last[1], bins='auto')
plt.show()
```

``` ipython
```

Biases
======

Serial Bias Curves
------------------

``` ipython
print(targ_ini.shape, rel_ini.shape, ref_ini.shape, end_point_ini.shape)
```

``` ipython
n_bins = 8
data_ini = pd.DataFrame({'target_loc': targ_ini[:, -1], 'rel_loc': rel_ini[:, -1], 'ref_loc': ref_ini[:, -1], 'errors': end_point_ini[1]})
data_last = pd.DataFrame({'target_loc': targ_last[:, -1], 'rel_loc': rel_last[:, -1], 'ref_loc': ref_last[:, -1], 'errors': end_point_last[1]})
```

``` ipython
def get_correct_error(n_bins, df, error_type='rel_loc', thresh=25):
    import numpy as np
    import pandas as pd

    # 1. Threshold errors
    if thresh is not None:
        data = df[(df['errors'] >= -thresh) & (df['errors'] <= thresh)].copy()
    else:
        data = df.copy()

    # 2. Bin target locations
    bin_edges = np.linspace(0, 360, n_bins + 1)
    data['bin_target'] = pd.cut(data['target_loc'], bins=bin_edges, include_lowest=True)
    mean_errors_per_bin = data.groupby('bin_target')['errors'].mean()

    # 3. Remove mean error per target location (for rel_loc)
    if error_type == 'rel_loc':
        data['adjusted_errors'] = data['errors'] - data['bin_target'].map(mean_errors_per_bin).astype(float)
    else:
        data['adjusted_errors'] = data['errors']

    # 4. Bin by error_type for both full versions
    data['bin_error'] = pd.cut(data[error_type], bins=n_bins)
    bin_error = data.groupby('bin_error')['adjusted_errors'].agg(['mean', 'sem']).reset_index()
    edges = bin_error['bin_error'].cat.categories
    centers = (edges.left + edges.right) / 2

    # 5. Flipped error absolute analysis
    if error_type == 'rel_loc':
        # Bin abs(rel_loc) from 0 to 180
        data['error_abs'] = np.abs(data[error_type])
        data['bin_error_abs'] = pd.cut(data['error_abs'], bins=n_bins, include_lowest=True)
        # Flip so all directions use same sign
        data['adjusted_errors_abs'] = data['adjusted_errors'] * np.sign(data[error_type])
    else:
        # Bin abs(ref_loc) from 0 to 90
        data['error_abs'] = np.abs(data[error_type])
        data = data[data['error_abs'] <= 90.0]  # Only 0-90
        data['bin_error_abs'] = pd.cut(data['error_abs'], bins=n_bins, include_lowest=True)
        # Flip so all directions use same sign for ref_loc
        data['adjusted_errors_abs'] = data['adjusted_errors'] * np.sign(data[error_type])

    bin_error_abs = data.groupby('bin_error_abs')['adjusted_errors_abs'].agg(['mean', 'sem']).reset_index()
    edges_abs = bin_error_abs['bin_error_abs'].cat.categories
    centers_abs = (edges_abs.left + edges_abs.right) / 2

    return centers, bin_error, centers_abs, bin_error_abs
```

``` ipython
centers_ini, bin_rel_ini, centers_abs_ini, bin_rel_abs_ini = get_correct_error(n_bins, data_ini)
centers_last, bin_rel_last, centers_abs_last, bin_rel_abs_last = get_correct_error(n_bins, data_last)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ini, bin_rel_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ini, bin_rel_ini['mean'] - bin_rel_ini['sem'], bin_rel_ini['mean'] + bin_rel_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_last, bin_rel_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_last, bin_rel_last['mean'] - bin_rel_last['sem'], bin_rel_last['mean'] + bin_rel_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_abs_ini, bin_rel_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_abs_ini, bin_rel_abs_ini['mean'] - bin_rel_abs_ini['sem'], bin_rel_abs_ini['mean'] + bin_rel_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_abs_last, bin_rel_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_abs_last, bin_rel_abs_last['mean'] - bin_rel_abs_last['sem'], bin_rel_abs_last['mean'] + bin_rel_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Rel. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 180, 3))

plt.tight_layout()
plt.show()
```

``` ipython
centers_ref_ini, bin_ref_ini, centers_ref_abs_ini, bin_ref_abs_ini = get_correct_error(n_bins, data_ini, error_type='ref_loc')
centers_ref_last, bin_ref_last, centers_ref_abs_last, bin_ref_abs_last = get_correct_error(n_bins, data_last, error_type='ref_loc')
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ref_ini, bin_ref_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ref_ini, bin_ref_ini['mean'] - bin_ref_ini['sem'], bin_ref_ini['mean'] + bin_ref_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_ref_last, bin_ref_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_ref_last, bin_ref_last['mean'] - bin_ref_last['sem'], bin_ref_last['mean'] + bin_ref_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Ref. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_ref_abs_ini, bin_ref_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_ref_abs_ini, bin_ref_abs_ini['mean'] - bin_ref_abs_ini['sem'], bin_ref_abs_ini['mean'] + bin_ref_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_ref_abs_last, bin_ref_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_ref_abs_last, bin_ref_abs_last['mean'] - bin_ref_abs_last['sem'], bin_ref_abs_last['mean'] + bin_ref_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Ref. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 90, 3))

plt.tight_layout()
plt.show()
```

``` ipython
```

Bias Evolution along a session
------------------------------

``` ipython
_, _, phi_list = decode_bump_torch(rates_list, axis=-1)
print(phi_list.shape)
```

``` ipython
print(phi0_list.shape, reference.shape)
```

``` ipython
cmap = plt.get_cmap('Blues')
colors = [cmap((i+1)/ phi_list.shape[0]) for i in range(phi_list.shape[0])]

n_bins = 8

serial_list = []
ref_list = []

for i in range(phi_list.shape[0]):
    targ_trial, rel_trial, ref_trial, errors_trial = get_rel_tar_error(phi_list[i, np.newaxis], phi0_list[i, np.newaxis], reference * np.pi / 180.0)
    end_point_trial = get_end_point(model, errors_trial)

    # print(targ_trial.shape, rel_trial.shape, ref_trial.shape, errors_trial.shape, end_point_trial.shape)

    data = pd.DataFrame({'target_loc': targ_trial[:, -1], 'rel_loc': rel_trial[:, -1], 'ref_loc': ref_trial[:, -1], 'errors': end_point_trial[1]})

    centers, bin_rel, centers_abs, bin_rel_abs = get_correct_error(n_bins, data)
    centers, bin_ref, centers_abs, bin_ref_abs = get_correct_error(n_bins, data, error_type='ref_loc')

    plt.plot(centers, bin_ref['mean'], color=colors[i])

    idx_max = np.argmax(abs(bin_rel_abs['mean']))
    serial_max = bin_rel_abs['mean'][idx_max]
    serial_std = bin_rel_abs['sem'][idx_max]

    serial_list.append([serial_max, serial_std])

    idx_max = np.argmax(abs(bin_ref_abs['mean']))
    ref_max = bin_ref_abs['mean'][idx_max]
    ref_std = bin_ref_abs['sem'][idx_max]

    ref_list.append([ref_max, ref_std])

serial_list = np.array(serial_list).T
ref_list = np.array(ref_list).T
print(serial_list.shape)
```

``` ipython
delay_duration = 100
xdelay = np.linspace(0, delay_duration, serial_list.shape[1])

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

from scipy.ndimage import gaussian_filter1d


s0 = 5
ax[0].plot(xdelay, gaussian_filter1d(serial_list[0], s0), '-')
ax[0].fill_between(xdelay, gaussian_filter1d(serial_list[0] - serial_list[1], s0), gaussian_filter1d(serial_list[0] + serial_list[1], s0), color='b', alpha=0.2)
ax[0].axhline(0, ls='--', color='k')

ax[0].set_xlabel('Trial #')
ax[0].set_ylabel('Serial Bias (°)')

ax[1].plot(xdelay, gaussian_filter1d(ref_list[0], s0), '-')
ax[1].fill_between(xdelay, gaussian_filter1d(ref_list[0] - ref_list[1], s0), gaussian_filter1d(ref_list[0] + ref_list[1], s0), color='b', alpha=0.2)
ax[1].axhline(0, ls='--', color='k')

ax[1].set_xlabel('Trial #')
ax[1].set_ylabel('Ref. Bias (°)')

plt.show()
```

``` ipython
```

Decoder
=======

Pref loc
--------

``` ipython
from astropy.stats.circstats import circmean

def get_pref_locs(nbins, mean_rates, angles):
    pref_locs = []

    bins = np.linspace(0, 2*np.pi, nbins + 1)
    counts, _ = np.histogram(angles, bins=bins)
    bin_indices = np.digitize(angles, bins) - 1

    normalized_rates = np.zeros_like(mean_rates, dtype=float)

    for i in range(mean_rates.shape[0]):
            bin_index = bin_indices[i]
            if 0 <= bin_index < nbins:  # Ensure index is within valid range
                    normalized_rates[i] = mean_rates[i] / counts[bin_index] if counts[bin_index] > 0 else 0

    pref_locs = []
    for i in range(mean_rates.shape[1]):
            pref_locs.append(circmean(angles, weights=normalized_rates[:, i], axis=0))

    pref_locs = np.array(pref_locs) # + 2.0 * np.pi) % (2.0 * np.pi)

    return pref_locs
```

``` ipython
from astropy.stats.circstats import circmean
import numpy as np

def get_pref_locs(nbins, mean_rates, angles):
    # Bin the angles
    bins = np.linspace(0, 2*np.pi, nbins + 1)
    bin_indices = np.digitize(angles, bins) - 1

    # For each bin: how many samples fall there
    counts = np.bincount(bin_indices, minlength=nbins)
    # Avoid division by zero
    with np.errstate(divide='ignore', invalid='ignore'):
        weights = 1.0 / np.where(counts > 0, counts, 1)  # [nbins]
        weights[counts == 0] = 0

    # For each sample, apply weight by the bin it's in:
    sample_weights = weights[bin_indices]  # [n_samples]

    # Now, for each neuron:
    pref_locs = []
    for n in range(mean_rates.shape[1]):
        # Weight mean_rates[:, n] by sample_weights
        w = mean_rates[:, n] * sample_weights
        # circmean over all angles, weighted
        mu = circmean(angles, weights=w, axis=0)
        pref_locs.append(mu % (2*np.pi))
    return np.array(pref_locs)
```

``` ipython
print(rates_list.shape, phi0_list.shape)
stim_start_idx = ((model.start_indices - model.N_STEADY) / model.N_WINDOW).to(int).cpu().numpy()
stim_end_idx = ((model.end_indices - model.N_STEADY) / model.N_WINDOW).to(int).cpu().numpy()
print(stim_start_idx.shape)
```

``` ipython
print(phi0_list.shape)
phi0_ini = np.vstack(phi0_list[:n_half, ..., 0])
phi0_last = np.vstack(phi0_list[-n_half:, ..., 0])
print(phi0_ini.shape)
```

``` ipython
mean_rates_ini = np.vstack(rates_list[:n_half, :, stim_start_idx[2, 0]:stim_end_idx[2, 0], :].mean(2))
mean_rates_last = np.vstack(rates_list[-n_half:, :, stim_start_idx[2,0]:stim_end_idx[2, 0], :].mean(2))
print(mean_rates_ini.shape, phi0_ini.shape)
```

``` ipython
nbins = 120
pref_locs_ini = get_pref_locs(nbins, mean_rates_ini, phi0_ini[:, 2])
pref_locs_last = get_pref_locs(nbins, mean_rates_last, phi0_last[:, 2])
print(pref_locs_ini.shape, pref_locs_last.shape)
```

``` ipython
idx_ini = np.argsort(pref_locs_ini)
idx_last = np.argsort(pref_locs_last)
```

Decoding Bump Location
----------------------

``` ipython
from sklearn.neighbors import KernelDensity

def weights_from_pdf(angles_rad, bandwidth=0.5, beta=0.5):
    angles_rad_2d = angles_rad.reshape(-1, 1)
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(angles_rad_2d)
    log_dens = kde.score_samples(angles_rad_2d)
    densities = np.exp(log_dens)

    if beta == 0:
        weights = 1 / (np.exp(log_dens) + 1e-8)
        weights /= weights.mean()
    else:
        # Softmax-normalized inverse density (avoids extreme weights)
        # Temperature parameter: lower beta → more uniform weighting
        weights = np.exp(-beta * densities)
        weights = weights / weights.mean()  # Normalize

    return weights
```

``` ipython
def weights_from_hist(angles_rad, n_bins=32):
     hist, bin_edges = np.histogram(angles_rad, bins=n_bins)
     bin_indices = np.digitize(angles_rad, bins=bin_edges[:-1], right=True)

     weights = 1.0 / (np.sqrt(hist[bin_indices - 1]) + 1e-6)
     weights /= np.mean(weights)

     return weights
```

``` ipython
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np

class AngleDecoder(BaseEstimator, RegressorMixin):
    def __init__(self, penalty=None, scaler=True, class_weight=None):
        self.penalty = penalty
        self.scaler = scaler
        self.class_weight = class_weight
        self.reg_ = None
        self.pipe_ = None
        self._initialize_regressor()

    def _initialize_regressor(self):
        if self.penalty is None:
            self.reg_ = LinearRegression()
        elif self.penalty == 'l2':
            self.reg_ = RidgeCV()
        elif self.penalty == 'l1':
            self.reg_ = MultiTaskLassoCV()
        elif self.penalty == 'multi':
            self.reg_ = MultiOutputRegressor(LinearSVR())
        elif self.penalty == 'rbf':
            self.reg_ = MultiOutputRegressor(SVR(kernel='rbf', C=1e3, gamma=0.1))

        pipe = []
        if self.scaler:
            pipe.append(('scaler', StandardScaler()))

        pipe.append(('reg', self.reg_))

        self.pipe_ = Pipeline(pipe)

    def fit(self, X, y):
        Y = np.column_stack((np.cos(y), np.sin(y)))

        weights = None
        if self.class_weight=='balanced' or self.class_weight is not None:
            # weights = weights_from_hist(angles, n_bins=32)
            weights = weights_from_pdf(y, bandwidth=1.0, beta=0.1)

        self.pipe_.fit(X, Y, reg__sample_weight=weights)

        if self.penalty == 'rbf':
            pref_locs = np.nan
        else:
            pred_cos = self.pipe_.named_steps['reg'].coef_[0]
            pred_sin = self.pipe_.named_steps['reg'].coef_[1]
            pref_locs = np.arctan2(pred_sin, pred_cos)

        self.pref_locs_ = (pref_locs + 2.0 * np.pi) % (2.0 * np.pi)

        return self

    def predict(self, X):
        preds = self.pipe_.predict(X)
        pred_cos, pred_sin = preds[:, 0], preds[:, 1]
        pred_loc = np.arctan2(pred_sin, pred_cos)
        return (pred_loc + 2.0 * np.pi) % (2.0 * np.pi)
```

``` ipython
print('rates', rates_list.shape, 'phi0', phi0_list.shape)
```

Decoding trial by trial for all session

\#+begin<sub>x²src</sub> ipython from mne.decoding import
SlidingEstimator, GeneralizingEstimator from
sklearn.model<sub>selection</sub> import cross<sub>valpredict</sub>,
LeaveOneOut

regressor = AngleDecoder(penalty=None, scaler=True,
class<sub>weight</sub>='balanced') mne<sub>estimator</sub> =
SlidingEstimator(regressor, n<sub>jobs</sub>=-1, verbose=False)

half<sub>trial</sub> = rates<sub>list</sub>.shape\[2\] // 2

pred<sub>list</sub> = \[\] for epoch in tqdm(\[0, -1\]):

rates = rates<sub>list</sub>\[epoch, :,
:half<sub>trial</sub>\].swapaxes(1, 2) angles =
phi0<sub>list</sub>\[epoch, :, 0, 0\]

rates = rates<sub>list</sub>\[epoch, :,
half<sub>trial</sub>:\].swapaxes(1, 2) angles =
phi0<sub>list</sub>\[epoch, :, 2, 0\]

pred<sub>list</sub> = np.array(pred<sub>list</sub>) print('',
pred<sub>list</sub>.shape)

\#+end<sub>src</sub>

``` example
100% 2/2 [00:00<00:00, 35544.95it/s]
 (0,)
```

:

Decoding session by session for all trial

``` ipython
from mne.decoding import SlidingEstimator, GeneralizingEstimator
from sklearn.model_selection import cross_val_predict, LeaveOneOut

regressor = AngleDecoder(penalty=None, scaler=True, class_weight=True)
mne_estimator = SlidingEstimator(regressor, n_jobs=64, verbose=False)
# mne_estimator = GeneralizingEstimator(regressor, n_jobs=-1, verbose=False)

half_trial = rates_list.shape[2] // 2
N_SESSIONS = 768

pred_list = []
for session in tqdm(range(N_SESSIONS)):

    rates = rates_list[:, session, :half_trial].swapaxes(1, 2)
    angles = phi0_list[:, session, 0, 0]
    # pred_locs1 = cross_val_predict(mne_estimator, rates, angles, cv=5, n_jobs=64)

    rates = rates_list[:, session, half_trial:].swapaxes(1, 2)
    angles = phi0_list[:, session, 2, 0]
    # pred_locs2 = cross_val_predict(mne_estimator, rates, angles, cv=5, n_jobs=64)

    # pred_locs = np.concatenate((pred_locs1, pred_locs2), axis=-1)
    pred_locs = None
    pred_list.append(pred_locs)

pred_list = np.array(pred_list)
# pred_list = pred_list.swapaxes(0,1)
# pkl_save(pred_list, './pred_list')
print('\n', pred_list.shape)
```

:

``` ipython
pred_list = pkl_load('./pred_list')
```

``` ipython
import matplotlib.cm as cm
colors = cm.tab10.colors  # or cm.viridis.colors, etc.
xtime = np.linspace(0, model.DURATION, 121)

fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

for i in range(5):
    idx = np.random.randint(0, angles.shape[0])
    ax[0].plot(xtime, pred_list[0, idx] * 180 / np.pi, color=colors[i])

    ax[0].axhline(phi0_list[0, idx, 0, 0] * 180 / np.pi, xmax=0.5, ls='--', color=colors[i])
    ax[0].axhline(phi0_list[0, idx, 2, 0] * 180 / np.pi, xmin=0.5, ls='--', color=colors[i])

add_vlines(model, ax[0])
ax[0].set_ylabel('Prediction (°)')
ax[0].set_xlabel('Time (s)')

error_prev = (pred_list[0, :] - phi0_list[0, :N_SESSIONS, 0]).T
error_prev = (error_prev + np.pi) % (2 * np.pi) - np.pi
error_prev *= 180 / np.pi

ax[1].plot(xtime, error_prev, alpha=0.5)
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Prev. Error (°)')

error_curr = (pred_list[0, :] - phi0_list[0, :N_SESSIONS, 2]).T
error_curr = (error_curr + np.pi) % (2 * np.pi) - np.pi
error_curr *= 180 / np.pi

ax[2].plot(xtime, error_curr, alpha=0.5)
ax[2].set_xlabel('Time (s)')
ax[2].set_ylabel('Curr. Error (°)')

plt.show()
```

``` ipython
```

Errors
------

``` ipython
n_half = num_epochs // 2
# n_half = 5

phi0_ini =  phi0_list[:n_half, :N_SESSIONS]
phi0_last = phi0_list[-n_half:, :N_SESSIONS]

phi_ini =  (pred_list[:n_half])
phi_last = (pred_list[-n_half:])

refs = reference[:N_SESSIONS] * np.pi / 180.0
print(phi_ini.shape, phi0_last.shape, refs.shape)
```

``` ipython
targ_ini, rel_ini, ref_ini, errors_ini = get_rel_tar_error(phi_ini, phi0_ini, refs)
targ_last, rel_last, ref_last, errors_last = get_rel_tar_error(phi_last, phi0_last, refs)
```

``` ipython
print(ref_ini.shape, rel_ini.shape, errors_ini.shape)
plt.plot(errors_ini[0, 1, :20].T)
plt.show()
```

``` ipython
end_point_ini = get_end_point(model, errors_ini)
end_point_last = get_end_point(model, errors_last)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

ax[0].hist(reference[:, 0])
ax[0].set_xlabel('References')

ax[1].hist(end_point_ini[1], bins=100)
ax[1].set_xlabel('Prev. Error (°)')

ax[2].hist(end_point_last[1], bins=100)
ax[2].set_xlabel('Curr. Error (°)')

plt.show()
```

``` ipython
```

Biases
------

``` ipython
print(targ_ini.shape, rel_ini.shape, ref_ini.shape, end_point_ini.shape)
```

``` ipython
n_bins = 8
data_ini = pd.DataFrame({'target_loc': targ_ini[:, -1], 'rel_loc': rel_ini[:, -1], 'ref_loc': ref_ini[:, -1], 'errors': end_point_ini[1]})
data_last = pd.DataFrame({'target_loc': targ_last[:, -1], 'rel_loc': rel_last[:, -1], 'ref_loc': ref_last[:, -1], 'errors': end_point_last[1]})
```

``` ipython
centers_ini, bin_rel_ini, centers_abs_ini, bin_rel_abs_ini = get_correct_error(n_bins, data_ini)
centers_last, bin_rel_last, centers_abs_last, bin_rel_abs_last = get_correct_error(n_bins, data_last)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ini, bin_rel_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ini, bin_rel_ini['mean'] - bin_rel_ini['sem'], bin_rel_ini['mean'] + bin_rel_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_last, bin_rel_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_last, bin_rel_last['mean'] - bin_rel_last['sem'], bin_rel_last['mean'] + bin_rel_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Rel. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_abs_ini, bin_rel_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_abs_ini, bin_rel_abs_ini['mean'] - bin_rel_abs_ini['sem'], bin_rel_abs_ini['mean'] + bin_rel_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_abs_last, bin_rel_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_abs_last, bin_rel_abs_last['mean'] - bin_rel_abs_last['sem'], bin_rel_abs_last['mean'] + bin_rel_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Rel. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 180, 3))

plt.tight_layout()
plt.show()
```

``` ipython
centers_ref_ini, bin_ref_ini, centers_ref_abs_ini, bin_ref_abs_ini = get_correct_error(n_bins, data_ini, error_type='ref_loc')
centers_ref_last, bin_ref_last, centers_ref_abs_last, bin_ref_abs_last = get_correct_error(n_bins, data_last, error_type='ref_loc')
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

ax[0].plot(centers_ref_ini, bin_ref_ini['mean'], 'r', label='First half')
ax[0].fill_between(centers_ref_ini, bin_ref_ini['mean'] - bin_ref_ini['sem'], bin_ref_ini['mean'] + bin_ref_ini['sem'], color='r', alpha=0.2)

ax[0].plot(centers_ref_last, bin_ref_last['mean'], 'b', label='Last half')
ax[0].fill_between(centers_ref_last, bin_ref_last['mean'] - bin_ref_last['sem'], bin_ref_last['mean'] + bin_ref_last['sem'], color='b', alpha=0.2)

ax[0].axhline(0, color='k', linestyle=":")
ax[0].set_xlabel('Ref. Loc. (°)')
ax[0].set_ylabel('Error (°)')
ax[0].set_xticks(np.linspace(-180, 180, 5))

ax[1].plot(centers_ref_abs_ini, bin_ref_abs_ini['mean'], 'r', label='First half')
ax[1].fill_between(centers_ref_abs_ini, bin_ref_abs_ini['mean'] - bin_ref_abs_ini['sem'], bin_ref_abs_ini['mean'] + bin_ref_abs_ini['sem'], color='r', alpha=0.2)

ax[1].plot(centers_ref_abs_last, bin_ref_abs_last['mean'], 'b', label='Last half')
ax[1].fill_between(centers_ref_abs_last, bin_ref_abs_last['mean'] - bin_ref_abs_last['sem'], bin_ref_abs_last['mean'] + bin_ref_abs_last['sem'], color='b', alpha=0.2)

ax[1].axhline(0, color='k', linestyle=":")
ax[1].set_xlabel('Ref. Loc. (°)')
ax[1].set_ylabel('Flip. Error (°)')
ax[1].legend(fontsize=12)
ax[1].set_xticks(np.linspace(0, 90, 3))

plt.tight_layout()
plt.show()
```

``` ipython
```

Bias Evolution along a session
------------------------------

``` ipython
#_, _, phi_list = decode_bump_torch(rates_list, axis=-1)
# print(phi_list.shape)
```

``` ipython
print(phi0_list.shape, reference.shape)
```

``` ipython
n_bins = 8

serial_list = []
ref_list = []

for i in range(phi_list.shape[0]):
    targ_trial, rel_trial, ref_trial, errors_trial = get_rel_tar_error(pred_list[i, np.newaxis], phi0_list[i, np.newaxis], reference * np.pi / 180.0)
    end_point_trial = get_end_point(model, errors_trial)

    # print(targ_trial.shape, rel_trial.shape, ref_trial.shape, errors_trial.shape, end_point_trial.shape)

    data = pd.DataFrame({'target_loc': targ_trial[:, -1], 'rel_loc': rel_trial[:, -1], 'ref_loc': ref_trial[:, -1], 'errors': end_point_trial[1]})

    centers, bin_rel, centers_abs, bin_rel_abs = get_correct_error(n_bins, data)
    centers, bin_ref, centers_abs, bin_ref_abs = get_correct_error(n_bins, data, error_type='ref_loc')

    idx_max = np.argmax(abs(bin_rel_abs['mean']))
    serial_max = bin_rel_abs['mean'][idx_max]
    serial_std = bin_rel_abs['sem'][idx_max]

    serial_list.append([serial_max, serial_std])

    idx_max = np.argmax(abs(bin_ref_abs['mean']))
    ref_max = bin_ref_abs['mean'][idx_max]
    ref_std = bin_ref_abs['sem'][idx_max]

    ref_list.append([ref_max, ref_std])

serial_list = np.array(serial_list).T
ref_list = np.array(ref_list).T
print(serial_list.shape)
```

``` ipython
delay_duration = 100
xdelay = np.linspace(0, delay_duration, serial_list.shape[1])

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

from scipy.ndimage import gaussian_filter1d


s0 = 5
ax[0].plot(xdelay, gaussian_filter1d(serial_list[0], s0), '-')
ax[0].fill_between(xdelay, gaussian_filter1d(serial_list[0] - serial_list[1], s0), gaussian_filter1d(serial_list[0] + serial_list[1], s0), color='b', alpha=0.2)

ax[0].set_xlabel('Trial #')
ax[0].set_ylabel('Serial Bias (°)')

ax[1].plot(xdelay, gaussian_filter1d(ref_list[0], s0), '-')
ax[1].fill_between(xdelay, gaussian_filter1d(ref_list[0] - ref_list[1], s0), gaussian_filter1d(ref_list[0] + ref_list[1], s0), color='b', alpha=0.2)

ax[1].set_xlabel('Trial #')
ax[1].set_ylabel('Ref. Bias (°)')

plt.show()
```

``` ipython
```